In [165]:
# read data/gpw_data_copy.sqlite
import pandas as pd
import sqlite3
# Connect to the SQLite database
conn = sqlite3.connect('data/gpw_data.sqlite')


In [166]:
import pandas as pd
import plotly.express as px

# 0. Load & clean
shareholders = pd.read_sql_query("SELECT * FROM company_shareholders;", conn)
companies    = pd.read_sql_query("SELECT url, name, market_cap_mln FROM company_company;", conn)

shareholders['shares_pct'] = (
    shareholders['shares_pct']
      .str.replace('%','')
      .str.replace(',','.')
      .astype(float)
)
companies['market_cap_mln'] = (
    companies['market_cap_mln']
      .str.replace(' ','')
      .str.replace(',','.')
      .pipe(pd.to_numeric, errors='coerce')
)


In [167]:
# there is a typo in urls with double //. https://www.gpw.pl//spolka?isin=PLPGER000010
# remove it
companies['url'] = companies['url'].str.replace('//', '/', regex=False)

In [168]:
shareholders["company_url"] = shareholders["company_url"].str.replace('//', '/', regex=False)

In [170]:

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Data preparation as before
company_df = shareholders.merge(companies, left_on='company_url', right_on='url', how='left')
company_df = company_df[company_df['market_cap_mln'] > 0].copy()

company_df = company_df.drop_duplicates(subset=['company_url', 'name', 'shareholder'])

# add 'other' shareholder for missing percentage ownership
# first calculate shares_pct for known shareholders and then assign 100% - sum(shares_pct) to 'other' 
# add the row for 'other' shareholder
other_shareholders = company_df.groupby('name').apply(
    lambda x: pd.Series({
        'shareholder': 'Inni',
        'shares_pct': 100 - x['shares_pct'].sum(),
        'market_cap_mln': x['market_cap_mln'].iloc[0]  # use the first market cap value
    })
).reset_index()
# Append 'other' shareholders to the main DataFrame
company_df = pd.concat([company_df, other_shareholders], ignore_index=True)
# market_cap_mln  = market_cap_mln * shares_pct / 100

# Calculate total market cap per company 
company_df['total_market_cap_mln'] = company_df["market_cap_mln"]
company_df['market_cap_mln'] = company_df['total_market_cap_mln'] * company_df['shares_pct'] / 100
# Create dynamic thresholds: one per group of 20 companies
bucket_size = 20
thresholds = []
total_mkt = company_df.sort_values(by='total_market_cap_mln', ascending=False).reset_index(drop=True)
for i in range(0, len(total_mkt), bucket_size):
    bucket = total_mkt.iloc[i:i+bucket_size]
    thresholds.append(bucket['total_market_cap_mln'].min())
# Ensure thresholds are sorted ascending for correct filtering
thresholds = sorted(set(thresholds), reverse=False)

frames = []
slider_steps = []

for i, threshold in enumerate(thresholds):
    # Filter companies with market cap >= current threshold and < next threshold (if exists)
    if i < len(thresholds) - 1:
        filtered = company_df[
            (company_df['total_market_cap_mln'] >= threshold) &
            (company_df['total_market_cap_mln'] < thresholds[i + 1])
        ]
    else:
        filtered = company_df[company_df['total_market_cap_mln'] >= threshold]
    fig_tmp = px.treemap(
        filtered,
        path=['name', 'shareholder'],
        values='market_cap_mln',
        color='shares_pct',
        hover_data={
            'market_cap_mln': ':.2f',
            'shares_pct': ':.2f',
            'shareholder': True
        },
        custom_data=['shares_pct']
    )
    # Add percentage ownership to the texttemplate
    for d in fig_tmp.data:
        d.texttemplate = '%{label}<br>%{customdata[0]:.2f}% udział'
    frame = go.Frame(data=fig_tmp.data, name=str(int(threshold)))
    frames.append(frame)
    slider_steps.append({
        'method': 'animate',
        'label': f"{int(threshold)} mln+",
        'args': [
            [str(int(threshold))],
            {
                'mode': 'immediate',
                'frame': {'duration': 0, 'redraw': True},
                'transition': {'duration': 0}
            }
        ]
    })

# Initial figure = first threshold
initial_fig = px.treemap(
    company_df[company_df['market_cap_mln'] >= thresholds[0]],
    path=['name', 'shareholder'],
    values='market_cap_mln',
    color='shares_pct',
    hover_data={'market_cap_mln': ':.2f', 'shares_pct': ':.2f', 'shareholder': True},
    custom_data=['shares_pct'],
        width=1000,
    height=600,
)
for d in initial_fig.data:
    d.texttemplate = '%{label}<br>%{customdata[0]:.2f}% udział'

# Turn into go.Figure and add frames + slider
fig = go.Figure(
    data=initial_fig.data,
    layout=go.Layout(
        title="Struktura własnościowa spółek GPW (dynamiczne progi, max 20 na bucket)",
        sliders=[{
            'active': 0,
            'pad': {"t": 50},
            'currentvalue': {'prefix': 'Minimalna kapitalizacja: '},
            'steps': slider_steps
        }]
    ),
    frames=frames
)

fig.show()
fig.write_html("wlasnosc_gpw.html", include_plotlyjs="cdn", full_html=False)


/var/folders/1_/btj1k7sn3ls02qcwrbhlp5zc0000gp/T/ipykernel_34673/2309394659.py:15: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [171]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Data preparation as before
shareholders_df = shareholders.merge(companies, left_on='company_url', right_on='url', how='left')
shareholders_df['stake_value_mln'] = shareholders_df['shares_pct'] / 100 * shareholders_df['market_cap_mln']
shareholders_df = shareholders_df[shareholders_df['stake_value_mln'] > 0].copy()

# Calculate total stake value per shareholder
total_stake = shareholders_df.groupby('shareholder')['stake_value_mln'].sum().reset_index()
total_stake.columns = ['shareholder', 'total_stake_value_mln']

# Sort by total stake value descending
total_stake = total_stake.sort_values('total_stake_value_mln', ascending=False).reset_index(drop=True)

# Create dynamic thresholds: one per group of 20 shareholders
bucket_size = 20
thresholds = []
for i in range(0, len(total_stake), bucket_size):
    bucket = total_stake.iloc[i:i+bucket_size]
    thresholds.append(bucket['total_stake_value_mln'].min())
# Ensure thresholds are sorted descending for correct filtering
thresholds = sorted(set(thresholds), reverse=False)

# Merge back to get total stake value for each row
sh_df = shareholders_df.merge(total_stake, on='shareholder', how='left')
# Calculate percentage of stake in total stake for each row
sh_df['stake_pct_of_total'] = 100 * sh_df['stake_value_mln'] / sh_df['total_stake_value_mln']

frames2 = []
slider_steps2 = []

for i, threshold in enumerate(thresholds):
    # Filter shareholders with total stake value >= current threshold and < next threshold (if exists)
    if i < len(thresholds) - 1:
        filtered = sh_df[
            (sh_df['total_stake_value_mln'] >= threshold) &
            (sh_df['total_stake_value_mln'] < thresholds[i + 1])
        ]
    else:
        filtered = sh_df[sh_df['total_stake_value_mln'] >= threshold]

    fig_tmp = px.treemap(
        filtered,
        path=['shareholder', 'name'],
        values='stake_value_mln',
        color='shares_pct',
        hover_data={
            'stake_value_mln': ':.2f',
            'shares_pct': ':.2f',
            'market_cap_mln': ':.2f',
            'total_stake_value_mln': ':.2f',
            'stake_pct_of_total': ':.2f'
        },
        custom_data=['stake_pct_of_total']
    )
    # Add percentage of stake in total stake to the texttemplate
    for d in fig_tmp.data:
        d.texttemplate = '%{label}<br>%{value:.2f}mln<br>%{customdata[0]:.2f}% portfela'
    frame = go.Frame(data=fig_tmp.data, name=str(int(threshold)))
    frames2.append(frame)
    slider_steps2.append({
        'method': 'animate',
        'label': f"{int(threshold)} mln+",
        'args': [
            [str(int(threshold))],
            {
                'mode': 'immediate',
                'frame': {'duration': 0, 'redraw': True},
                'transition': {'duration': 0}
            }
        ]
    })

# Initial figure = first threshold
initial_fig2 = px.treemap(
    sh_df[sh_df['total_stake_value_mln'] >= thresholds[0]],
    path=['shareholder', 'name'],
    values='stake_value_mln',
    color='shares_pct',
    hover_data={
        'stake_value_mln': ':.2f',
        'shares_pct': ':.2f',
        'market_cap_mln': ':.2f',
        'total_stake_value_mln': ':.2f',
        'stake_pct_of_total': ':.2f'
    },

    custom_data=['stake_pct_of_total']
)
for d in initial_fig2.data:
    d.texttemplate = '%{label}<br>%{value:.2f}mln<br>%{customdata[0]:.2f}% portfela'

fig2 = go.Figure(
    data=initial_fig2.data,
    layout=go.Layout(
        title="Najwięksi akcjonariusze GPW i ich udziały (%) (dynamiczne progi, max 20 na bucket)",
        sliders=[{
            'active': 0,
            'pad': {"t": 50},
            'currentvalue': {'prefix': 'Minimalna wartość portfela: '},
            'steps': slider_steps2
        }]
    ),
    frames=frames2
)

fig2.show()
fig2.write_html("akcjonariusze_gpw.html", include_plotlyjs="cdn", full_html=False)

In [164]:
# see descriptions of companies

companies_desc    = pd.read_sql_query("SELECT name, description FROM company_company;", conn)
# pretty print
companies_desc['description'] = companies_desc['description'].str.replace('\n', ' ')
companies_desc['description'] = companies_desc['description'].str.replace('  ', ' ')
# Display descriptions
for index, row in companies_desc.iterrows():
    print(f"Company: {row['name']}\nDescription: {row['description']}\n")

Company: COALENERG
Description: Grupa specjalizuje się w wydobyciu, wzbogacaniu i sprzedaży węgla energetycznego i koksowego. Dostarcza surowiec do największych zakładów wytwórczych energii elektrycznej i ciepła oraz zakładów hutniczych na Ukrainie. Spółka eksportuje również węgiel do elektrowni w Turcji, Mołdawii, Bułgarii i Słowacji. Działalność grupy koncentruje się na efektywnym wydobyciu i dystrybucji węgla, odpowiadając na potrzeby energetyczne i przemysłowe w regionie.

Company: ALLEGRO
Description: Grupa jest właścicielem wiodącej platformy e-commerce w Polsce - Allegro.pl, która przyciąga miesięcznie około 20 milionów użytkowników. Portfolio spółki obejmuje również popularną porównywarkę cenową Ceneo.pl. Ponadto, grupa zarządza serwisem eBilet, specjalizującym się w sprzedaży biletów na różnorodne wydarzenia. Oferta grupy obejmuje szeroki zakres produktów i usług, w tym program lojalnościowy Allegro Smart! oraz usługi logistyczne Allegro Fulfillment. Spółka rozwija też sieć au